In [5]:
local std = require('std')

local https = require('ssl.https')
local cjson = require('cjson')

local API_ID = '64dd8f8c'
local API_KEY = '8f945b259bf29848031c8dbdb7abf4b3'

function request_api(stem)
    local url = string.format('https://api.tfl.gov.uk/%s&app_id=%s&app_key=%s', stem, API_ID, API_KEY)
    local body, code, _, _ = https.request(url)
    
    assert(code == 200, string.format('Invalid response to url %s. Code: %d', url, code))
        
    return cjson.decode(body)
end

function get_line_data()
    return request_api('Line?')
end
        
function get_lines_of_interest(line_data)
    local results = {}
    for k, v in pairs(line_data) do
        if v.modeName == 'tube' or v.modeName == 'overground' then table.insert(results, v.id) end
    end
    
    return results
end

function format_stem(...)
    local query = (...).query
    
    local stem = query .. '?'
    for k, v in pairs(...) do
        if k ~= 'query' then
            stem = string.format('%s&%s=%s', stem, k, v)
        end
    end
    
    return stem
end

function get_stations_for_line(name)
    local stem = format_stem{
        query='Line/ids/StopPoints',
        ids=name
    }

    return request_api(stem)
end

function get_station_names(station_data)
    local ids = {}
    for _, v in pairs(station_data) do
        ids[v.id] = v.commonName
    end
    return ids
end

function get_timetables(line_name, station_name, direction)
    stem = format_stem{
        query='Line/id/Timetable/fromStopPointId', 
        fromStopPointId=station_name, 
        id=line_name,
        direction=direction}
    
    return request_api(stem)
end

function get_intervals(timetable, stations)
    local routes = timetable.timetable.routes
    local results = {}
    for route_num, route in pairs(routes) do
        results[route_num] = {}
        for interval_num, intervals in pairs(route.stationIntervals) do
            results[route_num][interval_num] = {}
            for _, interval in pairs(intervals.intervals) do
                station_name = stations[interval.stopId]
                results[route_num][interval_num][station_name] = interval.timeToArrival
            end
        end
    end
    return results
end
    

function get_intervals_for_line(line_name, ignores)
    local ignores = ignores or {}
    local stations = get_station_names(get_stations_for_line(line_name))
    
    local results = {}
    for id, station_name in pairs(stations) do
        results[station_name] = {}
        for _, direction in pairs({'inbound', 'outbound'}) do
            ignore_this = (ignores[station_name] and ignores[station_name][direction])
            
            if not ignore_this then
                local success, timetables = pcall(get_timetables, line_name, id, direction)
            else
                print(string.format('Ignoring %s in direction %s', station_name, direction))
                local success = false
            end
                
            if success and not ignore_this then
                results[station_name][direction] = get_intervals(timetables, stations)
                print(string.format('Got results for %s in direction %s', station_name, direction))
            elseif not ignore_this then
                print(string.format('Failed to get intervals for %s in direction %s', station_name, direction))
            end
        end
    end
    
    return results
end

function save_intervals_for_all_lines()
    local lines_ = get_lines_of_interest(get_line_data())
        
    for line in std.ielems(lines_) do
        print(string.format('Getting information for line %s', line))
        local intervals = get_intervals_for_line(line)
        
        filename = string.format('lines/%s.t7', line)
        torch.save(filename, intervals)
    end
end

function update_intervals()
    local lines_ = get_lines_of_interest(get_line_data())

    for line in std.ielems(lines_) do
        print(string.format('Getting information for line %s', line))
        local filename = string.format('lines/%s.t7', line)
        local data = torch.load(filename)
        local new_data = get_intervals_for_line(line, data)
        for station_name, directions in pairs(new_data) do
            for direction_name, intervals in pairs(directions) do
                data[station_name][direction] = intervals
            end
        end
    
        torch.save(filename, data)
    end
end
    
update_intervals()
        

Getting information for line bakerloo	
Ignoring Harrow & Wealdstone Underground Station in direction inbound	
Failed to get intervals for Harrow & Wealdstone Underground Station in direction outbound	
Ignoring Kenton Underground Station in direction inbound	
Ignoring Kenton Underground Station in direction outbound	
Ignoring Maida Vale Underground Station in direction inbound	
Ignoring Maida Vale Underground Station in direction outbound	
Ignoring Regent's Park Underground Station in direction inbound	
Ignoring Regent's Park Underground Station in direction outbound	
Ignoring Paddington Underground Station in direction inbound	
Ignoring Paddington Underground Station in direction outbound	
Ignoring Harlesden Underground Station in direction inbound	
Ignoring Harlesden Underground Station in direction outbound	
Ignoring Lambeth North Underground Station in direction inbound	
Ignoring Lambeth North Underground Station in direction outbound	
Ignoring North Wembley Underground Station in d

Failed to get intervals for West Ruislip Underground Station in direction inbound	
Ignoring West Ruislip Underground Station in direction outbound	
Ignoring East Acton Underground Station in direction inbound	
Ignoring East Acton Underground Station in direction outbound	
Ignoring Holborn Underground Station in direction inbound	
Ignoring Holborn Underground Station in direction outbound	
Ignoring Theydon Bois Underground Station in direction inbound	
Ignoring Theydon Bois Underground Station in direction outbound	
Ignoring Chancery Lane Underground Station in direction inbound	
Ignoring Chancery Lane Underground Station in direction outbound	
Ignoring Woodford Underground Station in direction inbound	
Ignoring Woodford Underground Station in direction outbound	
Ignoring Bethnal Green Underground Station in direction inbound	
Ignoring Bethnal Green Underground Station in direction outbound	
Ignoring Tottenham Court Road Underground Station in direction inbound	
Ignoring Tottenham Court

Failed to get intervals for Hainault Underground Station in direction outbound	
Ignoring Ruislip Gardens Underground Station in direction inbound	
Ignoring Ruislip Gardens Underground Station in direction outbound	
Ignoring Queensway Underground Station in direction inbound	
Ignoring Queensway Underground Station in direction outbound	
Ignoring Redbridge Underground Station in direction inbound	
Ignoring Redbridge Underground Station in direction outbound	
Ignoring Debden Underground Station in direction inbound	
Ignoring Debden Underground Station in direction outbound	
Ignoring Holland Park Underground Station in direction inbound	
Ignoring Holland Park Underground Station in direction outbound	
Ignoring Epping Underground Station in direction inbound	


Failed to get intervals for Epping Underground Station in direction outbound	
Ignoring Newbury Park Underground Station in direction inbound	
Ignoring Newbury Park Underground Station in direction outbound	
Ignoring Hanger Lane Underground Station in direction inbound	
Ignoring Hanger Lane Underground Station in direction outbound	
Ignoring Gants Hill Underground Station in direction inbound	
Ignoring Gants Hill Underground Station in direction outbound	
Ignoring Liverpool Street Underground Station in direction inbound	
Ignoring Liverpool Street Underground Station in direction outbound	
Ignoring Loughton Underground Station in direction inbound	
Ignoring Loughton Underground Station in direction outbound	
Ignoring Marble Arch Underground Station in direction inbound	
Ignoring Marble Arch Underground Station in direction outbound	
Failed to get intervals for Ealing Broadway Underground Station in direction inbound	
Ignoring Ealing Broadway Underground Station in direction outbound	
Ig

Failed to get intervals for Westminster Underground Station in direction inbound	
Failed to get intervals for Westminster Underground Station in direction outbound	
Failed to get intervals for South Kensington Underground Station in direction inbound	


...

ection inbound	
Ignoring Chesham Underground Station in direction outbound	
Ignoring Rickmansworth Underground Station in direction inbound	
Ignoring Rickmansworth Underground Station in direction outbound	
Ignoring Pinner Underground Station in direction inbound	
Ignoring Pinner Underground Station in direction outbound	
Getting information for line northern	
Ignoring King's Cross St. Pancras Underground Station in direction inbound	
Ignoring King's Cross St. Pancras Underground Station in direction outbound	
Ignoring Tooting Broadway Underground Station in direction inbound	
Ignoring Tooting Broadway Underground Station in direction outbound	
Ignoring Edgware Underground Station in direction inbound	
Failed to get intervals for Edgware Underground Station in direction outbound	
Ignoring Old Street Underground Station in direction inbound	
Ignoring Old Street Underground Station in direction outbound	
Ignoring East Finchley Underground Station in direction inbound	
Ignoring East Finch

Failed to get intervals for Mill Hill East Underground Station in direction outbound	
Ignoring Totteridge & Whetstone Underground Station in direction inbound	
Ignoring Totteridge & Whetstone Underground Station in direction outbound	
Ignoring Clapham Common Underground Station in direction inbound	
Ignoring Clapham Common Underground Station in direction outbound	
Ignoring Stockwell Underground Station in direction inbound	
Ignoring Stockwell Underground Station in direction outbound	
Ignoring Kentish Town Underground Station in direction inbound	
Ignoring Kentish Town Underground Station in direction outbound	
Ignoring Mornington Crescent Underground Station in direction inbound	
Ignoring Mornington Crescent Underground Station in direction outbound	
Ignoring Moorgate Underground Station in direction inbound	
Ignoring Moorgate Underground Station in direction outbound	
Ignoring Colliers Wood Underground Station in direction inbound	
Ignoring Colliers Wood Underground Station in direc

Failed to get intervals for Heathrow Terminal 4 Underground Station in direction inbound	
Ignoring Heathrow Terminal 4 Underground Station in direction outbound	
Ignoring Earl's Court Underground Station in direction inbound	
Ignoring Earl's Court Underground Station in direction outbound	
Ignoring Ruislip Manor Underground Station in direction inbound	
Ignoring Ruislip Manor Underground Station in direction outbound	
Ignoring Ealing Common Underground Station in direction inbound	
Ignoring Ealing Common Underground Station in direction outbound	
Ignoring Hounslow East Underground Station in direction inbound	
Ignoring Hounslow East Underground Station in direction outbound	
Ignoring Arsenal Underground Station in direction inbound	
Ignoring Arsenal Underground Station in direction outbound	
Ignoring Barons Court Underground Station in direction inbound	
Ignoring Barons Court Underground Station in direction outbound	
Ignoring Wood Green Underground Station in direction inbound	
Ignori

ing Piccadilly Circus Underground Station in direction outbound	
Ignoring Caledonian Road Underground Station in direction inbound	
Ignoring Caledonian Road Underground Station in direction outbound	
Ignoring Cockfosters Underground Station in direction inbound	
Failed to get intervals for Cockfosters Underground Station in direction outbound	
Ignoring Northfields Underground Station in direction inbound	
Ignoring Northfields Underground Station in direction outbound	
Ignoring Alperton Underground Station in direction inbound	
Ignoring Alperton Underground Station in direction outbound	
Ignoring Manor House Underground Station in direction inbound	
Ignoring Manor House Underground Station in direction outbound	
Ignoring Knightsbridge Underground Station in direction inbound	
Ignoring Knightsbridge Underground Station in direction outbound	
Ignoring Boston Manor Underground Station in direction inbound	
Ignoring Boston Manor Underground Station in direction outbound	
Ignoring Eastcote U

cus Underground Station in direction inbound	
Ignoring Oxford Circus Underground Station in direction outbound	
Ignoring Stockwell Underground Station in direction inbound	
Ignoring Stockwell Underground Station in direction outbound	
Ignoring Walthamstow Central Underground Station in direction inbound	
Failed to get intervals for Walthamstow Central Underground Station in direction outbound	
Getting information for line waterloo-city	
Ignoring Bank Underground Station in direction inbound	
Failed to get intervals for Bank Underground Station in direction outbound	
Failed to get intervals for Waterloo Underground Station in direction inbound	
Ignoring Waterloo Underground Station in direction outbound	


In [ ]:
require('lfs')
local std = require('std')

function load_intervals()
    local results = {}
    for filename in lfs.dir('lines') do
        if string.split(filename, '%.')[2] == 't7' then
            table.insert(results, torch.load('lines/' .. filename))
        end
    end
    
    return results
end

function flatten_to_lists(t)
    local is_leaf = type(t) == 'table' and type(std.table.values(t)[1]) ~= 'table'
    
    if is_leaf then
        local results = {}
        for key, value in pairs(t) do
            results[key] = {value}
        end
        return results
    else
        local results = {}
        for _, child in pairs(t) do
            for key, value in pairs(flatten_to_lists(child)) do
                if results[key] == nil then
                    results[key] = std.list.List()
                end
                
                results[key] = results[key] .. value
            end
        end
    
        return results
    end    
end
    

function gather_intervals_for_station(station_intervals)
    local intervals = {}
    for direction in std.elems(station_intervals) do
        for route in std.elems(direction) do
            for schedule in std.elems(route) do
                for destination, time in pairs(schedule) do
                    if intervals[destination] == nil then 
                        intervals[destination] = {time}
                    else
                        table.insert(intervals[destination], time)
                    end
                end
            end
        end
    end
    
    local results = {}
    for destination, destination_intervals in pairs(intervals) do
        results[destination] = torch.median(torch.Tensor(destination_intervals))[1]
    end
    
    return results
end
    
function gather_intervals_for_line(line_intervals)
    local results = {}
end

intervals = load_intervals()
--print(flatten_to_lists(intervals[1]['Maida Vale Underground Station']))
--print(flatten_to_lists({{{'a'}}, {{'b'}}}))
print(std.list.List{})

In [11]:
std = require('std')

data = torch.load('lines/bakerloo.t7')
if not data['Elephant & Castle Underground Station']['inbound'] then
    print('hello')
end

hello	
